In [12]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import matplotlib.pyplot as plt

In [13]:
import numpy as np
import torch.utils.data as utils

In [18]:
# The parts that you should complete are designated as TODO
class DeepNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        # TODO: define the layers of the network
        self.layer1 = nn.Linear(100,200)
        self.layer2 = nn.Linear(200, 300)
        self.layer3 = nn.Linear(300, 100)

    def forward(self, x):
        # TODO: define the forward pass of the network using the layers you defined in constructor
        x=F.sigmoid(self.layer1(x))
        x=F.sigmoid(self.layer2(x))
        x=F.sigmoid(self.layer3(x))
        return x


In [5]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    correct = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0: #Print loss every 100 batch
            print('Train Epoch: {}\tLoss: {:.6f}'.format(
                epoch, loss.item()))
    accuracy = test(model, device, train_loader)
    return accuracy

def test(model, device, test_loader):
    model.eval()
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    accuracy = 100. * correct / len(test_loader.dataset)

    return accuracy

In [ ]:
#Adam


def main():
    torch.manual_seed(1)
    np.random.seed(1)
    # Training settings
    use_cuda = False # Switch to False if you only want to use your CPU
    learning_rate = 0.001
    NumEpochs = 10
    batch_size = 32

    device = torch.device("cuda" if use_cuda else "cpu")

    train_X = np.load('Data/X_train.npy')
    train_Y = np.load('Data/y_train.npy')

    test_X = np.load('Data/X_test.npy')
    test_Y = np.load('Data/y_test.npy')

    train_X = train_X.reshape([-1,1,28,28]) # the data is flatten so we reshape it here to get to the original dimensions of images
    test_X = test_X.reshape([-1,1,28,28])

    # transform to torch tensors
    tensor_x = torch.tensor(train_X, device=device)
    tensor_y = torch.tensor(train_Y, dtype=torch.long, device=device)

    test_tensor_x = torch.tensor(test_X, device=device)
    test_tensor_y = torch.tensor(test_Y, dtype=torch.long)

    train_dataset = utils.TensorDataset(tensor_x,tensor_y) # create your datset
    train_loader = utils.DataLoader(train_dataset, batch_size=batch_size) # create your dataloader

    test_dataset = utils.TensorDataset(test_tensor_x,test_tensor_y) # create your datset
    test_loader = utils.DataLoader(test_dataset) # create your dataloader if you get a error when loading test data you can set a batch_size here as well like train_dataloader

    model = ConvNet().to(device)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate, betas=(0.999,0.999))
    
    Acc_train=torch.zeros(NumEpochs)
    Acc_test=torch.zeros(NumEpochs)

    for epoch in range(NumEpochs):
        train_acc = train(model, device, train_loader, optimizer, epoch)
        print('\nTrain set Accuracy: {:.0f}%\n'.format(train_acc))
        Acc_train[epoch]=train_acc
        test_acc = test(model, device, test_loader)
        print('\nTest set Accuracy: {:.0f}%\n'.format(test_acc))
        Acc_test[epoch]=test_acc

    torch.save(model.state_dict(), "mnist_cnn.pt")

    #TODO: Plot train and test accuracy vs epoch
    epoch_axis=torch.tensor([i+1 for i in range(NumEpochs) ])
    plt.xlabel('The number of Epochs')
    plt.ylabel('Accuracy in percentage')
    plt.title('Adam with learning rate 0.001, betas=(0.999,0.999)')
    plt.plot(epoch_axis,Acc_train, label='train accuracy')
    plt.plot(epoch_axis,Acc_test, label='test accuracy')
    plt.legend()
    plt.show()


if __name__ == '__main__':
    main()
